In [1]:
!pip install langchain langchain-community langchain-text-splitters langchain-chroma chromadb sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 65.8 MB/s eta 0:00:

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma


In [3]:
import pandas as pd

books = pd.read_csv('/content/my_dataset.csv', engine='python', quotechar='"', escapechar='\\')
books.head()


,title,year,tags,tagged_description,isbn13
0,The Master of Diabolism,2017,"['Action', 'Adventure', 'BL', 'Comedy', 'Manhu...",9780000000001 As the grandmaster who founded t...,9780000000001
1,JoJo's Bizarre Adventure Part 7: Steel Ball Run,2004,"['Action', 'Adventure', 'Horror', 'Mystery', '...","9780000000002 Set in 1890, Steel Ball Run spot...",9780000000002
2,A Sign of Affection,2019,"['Romance', 'Shoujo', 'Slice of Life', 'Disabi...",9780000000003 Yuki is a typical college studen...,9780000000003
3,Moriarty the Patriot,2016,"['Mystery', 'Shounen', 'Detectives', 'England'...","9780000000004 Before he was Sherlock’s rival, ...",9780000000004
4,Classroom of the Elite (Light Novel),2015,"['Comedy', 'Drama', 'Ecchi', 'Harem', 'Light N...",9780000000005 At the prestigious Tokyo Metropo...,9780000000005


In [4]:
books["tagged_description"]

,tagged_description
0,9780000000001 As the grandmaster who founded t...
1,"9780000000002 Set in 1890, Steel Ball Run spot..."
2,9780000000003 Yuki is a typical college studen...
3,"9780000000004 Before he was Sherlock’s rival, ..."
4,9780000000005 At the prestigious Tokyo Metropo...
...,...
42264,9780000070937 Following a life of bloodshed an...
42265,9780000070938 When her father squanders the fa...
42266,"9780000070939 Hiyu, a college student who live..."
42267,9780000070941 When the weak and unconfident Ch...


In [6]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   sep = "\n",
                                   index = False,
                                   header = False)

In [7]:
raw_documents = TextLoader("tagged_description.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)


Streaming output truncated to the last 5000 lines.


In [8]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='9780000000001 As the grandmaster who founded the Demonic Sect, Wei WuXian roamed the world in his wanton ways, hated by millions for the chaos he created. In the end, he was back-stabbed by his dearest shidi and killed by powerful clans that combined to overpower him. He incarnates into the body of a lunatic who was abandoned by his clan and is later, unwillingly, taken away by a famous cultivator among the clans—Lan WangJi, his archenemy.')

In [9]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db_books = Chroma.from_documents(
    documents,
    embedding=embedding_model
)

<ipython-input-9-90f963e0fd8b>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
query = "A manga about power in a ninja world "
docs = db_books.similarity_search(query, k = 10)
docs

[Document(id='29724ca5-4cff-4e71-9a78-73375670536e', metadata={'source': 'tagged_description.txt'}, page_content='"9780000059399 This is where a new ninja story begins. With the end of the samurai era, the role of the ninjas came to an end.... However, in preparation for an era when they might be needed again, they continued to hide out and secretly hone their skills in various parts of the world.Ninja World"" is an epic tale of ninjas working in secret around the world. The first issue takes place in the United States. How will one ninja fight in the gun-toting land of America? Ninjas are real!"'),
 Document(id='a9806176-d1d9-4ad9-a7af-36713bbf2947', metadata={'source': 'tagged_description.txt'}, page_content='9780000018879 It’s set in feudal japan and its central character is an insane sadistic princess with a elite unit of special “hair” ninjas. She gets reckless and draws the attention of the Shogun which leads to a huge clash.'),
 Document(id='090f94d2-33ec-40c0-bad4-ebde41498476'

In [12]:
books.head()

,title,year,tags,tagged_description,isbn13
0,The Master of Diabolism,2017,"['Action', 'Adventure', 'BL', 'Comedy', 'Manhu...",9780000000001 As the grandmaster who founded t...,9780000000001
1,JoJo's Bizarre Adventure Part 7: Steel Ball Run,2004,"['Action', 'Adventure', 'Horror', 'Mystery', '...","9780000000002 Set in 1890, Steel Ball Run spot...",9780000000002
2,A Sign of Affection,2019,"['Romance', 'Shoujo', 'Slice of Life', 'Disabi...",9780000000003 Yuki is a typical college studen...,9780000000003
3,Moriarty the Patriot,2016,"['Mystery', 'Shounen', 'Detectives', 'England'...","9780000000004 Before he was Sherlock’s rival, ...",9780000000004
4,Classroom of the Elite (Light Novel),2015,"['Comedy', 'Drama', 'Ecchi', 'Harem', 'Light N...",9780000000005 At the prestigious Tokyo Metropo...,9780000000005


In [13]:
isbn_to_find = '9780000059399'
result = books[books['isbn13'] == isbn_to_find]

# Display result
print(result)


Empty DataFrame
Columns: [title, year, tags, tagged_description, isbn13]
Index: []


In [14]:
books['isbn13']

,isbn13
0,9780000000001
1,9780000000002
2,9780000000003
3,9780000000004
4,9780000000005
...,...
42264,9780000070937
42265,9780000070938
42266,9780000070939
42267,9780000070941


In [15]:
books


,title,year,tags,tagged_description,isbn13
0,The Master of Diabolism,2017,"['Action', 'Adventure', 'BL', 'Comedy', 'Manhu...",9780000000001 As the grandmaster who founded t...,9780000000001
1,JoJo's Bizarre Adventure Part 7: Steel Ball Run,2004,"['Action', 'Adventure', 'Horror', 'Mystery', '...","9780000000002 Set in 1890, Steel Ball Run spot...",9780000000002
2,A Sign of Affection,2019,"['Romance', 'Shoujo', 'Slice of Life', 'Disabi...",9780000000003 Yuki is a typical college studen...,9780000000003
3,Moriarty the Patriot,2016,"['Mystery', 'Shounen', 'Detectives', 'England'...","9780000000004 Before he was Sherlock’s rival, ...",9780000000004
4,Classroom of the Elite (Light Novel),2015,"['Comedy', 'Drama', 'Ecchi', 'Harem', 'Light N...",9780000000005 At the prestigious Tokyo Metropo...,9780000000005
...,...,...,...,...,...
42264,The Lady and the Beast (Novel),2020,"['Fantasy', 'Manhwa', 'Romance', 'Web Novels']",9780000070937 Following a life of bloodshed an...,9780000070937
42265,The Boomerang Duke (Novel),2020,"['Comedy', 'Manhwa', 'Romance', 'Web Novels', ...",9780000070938 When her father squanders the fa...,9780000070938
42266,"Atashi no Senzoku, Mahoutsukai-sama!",2021,"['Romance', 'Shoujo', 'Supernatural']","9780000070939 Hiyu, a college student who live...",9780000070939
42267,Necromancer Survival,2022,"['BL', 'Comedy', 'Drama', 'Fantasy', 'Manhwa',...",9780000070941 When the weak and unconfident Ch...,9780000070941


In [20]:
isbn_to_find = "9780000000001"  # Replace with the ISBN you are looking for
isbn_to_find = int(isbn_to_find)  # Convert to int if necessary

result = books[books['isbn13'] == isbn_to_find]

# Display result
print(result)


                     title  year  \
0  The Master of Diabolism  2017   

                                                tags  \
0  ['Action', 'Adventure', 'BL', 'Comedy', 'Manhu...   

                                  tagged_description         isbn13  
0  9780000000001 As the grandmaster who founded t...  9780000000001  


In [23]:
books[books['isbn13'] == docs[0].page_content.split()[0].strip(' "')]

,title,year,tags,tagged_description,isbn13


In [24]:
print(books['isbn13'].dtype)  # Check the type of isbn13 column
print(type(docs[0].page_content.split()[0].strip('"')))  # Check the type of extracted value


int64
<class 'str'>


In [25]:
books['isbn13'] = books['isbn13'].astype(str)


In [26]:
isbn_to_find = docs[0].page_content.split()[0].strip('"')  # Your extracted value
result = books[books['isbn13'] == isbn_to_find]
print(result)


                 title  year                           tags  \
37207  Ninja World USA  2020  ['Drama', 'America', 'Ninja']   

                                      tagged_description         isbn13  
37207  9780000059399 This is where a new ninja story ...  9780000059399  


In [27]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k=50)

    books_list = [recs[i].page_content.strip('"').split()[0] for i in range(len(recs))]

    return books[books["isbn13"].isin(books_list)]


In [30]:
retrieve_semantic_recommendations("")

,title,year,tags,tagged_description,isbn13
664,Love Me for Who I Am,2018,"['Drama', 'Romance', 'Cafe', 'Crossdressing', ...",9780000000738 Mogumo is a cute but lonely high...,9780000000738
1469,My Lesbian Experience With Loneliness,2016,"['Drama', 'GL', 'Slice of Life', 'Yuri', 'Adul...",9780000001654 It tells a personal story of an ...,9780000001654
2700,The Erotic Way Of The Manga Artist: Studying Y...,2014,"['BL', 'Yaoi', 'Adult Couples', 'Adult Industr...",9780000003068 Through physical gay sex I'm tra...,9780000003068
2791,IS - Aiesu,2003,"['Drama', 'Josei', 'Romance', 'LGBT Themes', '...",9780000003180 This manga deals with very serio...,9780000003180
3155,Over-Cumming Writer's Block,2017,"['Comedy', 'Romance', 'Shoujo', 'Smut', 'Write...",9780000003587 Romance novelist Rio's strugglin...,9780000003587
3382,Romantic Bitch,2017,"['BL', 'Yaoi', 'Adult Couples', 'Bara']","9780000003850 ""Your body is lewd. But... you h...",9780000003850
5519,I Don't Know How to Give Birth!,2015,"['Comedy', 'Slice of Life', 'Childcare', 'Comi...",9780000006339 The humorous and heartfelt autob...,9780000006339
6447,The Artist and the Beast,2015,"['Comedy', 'Josei', 'Romance', 'Adult Couples'...",9780000007444 Annoyed by everything and lackin...,9780000007444
6891,Naked,2002,"['Shoujo', 'School Life']",9780000007987 Naked is a one-volume manga cont...,9780000007987
7166,Boku ga Watashi ni Naru Tame ni,2016,"['Seinen', 'Slice of Life', 'Autobiographies',...",9780000008318 An autobiographical manga that t...,9780000008318
